Teams Data Scraping

In [1]:
import requests
from bs4 import BeautifulSoup 
import json
import pandas as pd
import os

In [8]:
base_url = 'https://understat.com/league/EPL/'
season_names = ['2014', '2015', '2016', '2017', '2018','2019']


def season_scraper(season):
    url = base_url + season
    res = requests.get(url)
    soup = BeautifulSoup(res.content, 'lxml')
    
    # data is under "scripts" tag. So, finding all the script tags
    script = soup.find_all('script')
    
    string_with_json_obj = ''

    for el in script:
        if 'teamsData' in el.text:
            string_with_json_obj = el.text.strip()
            
    #keeping only the part of string we need
    ind_start = string_with_json_obj.index("('") + 2
    ind_end = string_with_json_obj.index("')") 
    json_data = string_with_json_obj[ind_start: ind_end]

    json_data = json_data.encode('utf8').decode('unicode_escape')
    data = json.loads(json_data)#formatted json data
    
    #making a dictionary composed opf team ID and team titles
    teams ={}
    for id in data.keys():
        teams[id] = data[id]['title']
        
    #getting column names 
    columns = []
    values = []
    for id in data.keys():
        columns = list(data[id]['history'][0].keys())
        break
    all_team_dicts = {}
    
    #getting all data for each team in the season and storing them in a dictionart with 'team' as key
    for id, team in teams.items():
        teams_data = []
        for i in data[id]['history']:
            teams_data.append(list(i.values()))

        df = pd.DataFrame(teams_data, columns = columns)
        all_team_dicts[team] = df
    #correcting the format of PPDA and OPDA(PPDA_allowed) to the ratio of PPDA and PPDA_allowed
    for team, df in all_team_dicts.items():
        all_team_dicts[team]['ppda_coef'] = all_team_dicts[team]['ppda'].apply(lambda x : x['att']/x['def'] if x['def'] != 0 else 0)
        all_team_dicts[team]['oppda_coef'] = all_team_dicts[team]['ppda_allowed'].apply(lambda x : x['att']/x['def'] if x['def'] != 0 else 0)